# Server-IP

Die nachfolgende Zeile ermittelt die öffentliche IP-Adresse eines Servers, der in einer Cloud-Umgebung betrieben wird, und speichert diese in einer Datei. 

Das Skript unterstützt verschiedene Cloud-Anbieter wie AWS, Azure und MAAS. 

Abhängig vom erkannten Cloud-Anbieter wird die jeweilige Methode zur Abfrage der öffentlichen IP-Adresse genutzt. Sollte der Cloud-Anbieter nicht erkannt werden, wird die IP-Adresse basierend auf der Netzwerkschnittstelle des Servers ermittelt. 

Die ermittelte IP-Adresse wird dann in der Datei server-ip im Verzeichnis ~/work/ gespeichert, wo sie die anderen Notebooks wieder ansprechen.

In [ ]:
%%bash
# Public IP anhand Cloud Provider setzen, WireGuard ueberschreibt alle
cloud_provider=$(cloud-init query v1.cloud_name 2>/dev/null) 
case "$cloud_provider" in
      "aws")
        public_ip=$(sudo cloud-init query ds.meta_data.public_hostname)
        ;;
      "gce" | "gcloud")
        public_ip=$(curl -s "http://metadata.google.internal/computeMetadata/v1/instance/network-interfaces/0/access-configs/0/external-ip" -H "Metadata-Flavor: Google")
        ;;  
    "azure")
        public_ip=$(jq -r '.ds.meta_data.imds.network.interface[0].ipv4.ipAddress[0].publicIpAddress' /run/cloud-init/instance-data.json 2>/dev/null)
        ;;
    "maas")
        public_ip=$(hostname).maas
        ;;        
    *)
        public_ip=$(hostname -I | cut -d ' ' -f 1) 
        ;;
esac
echo $public_ip >~/work/server-ip

# aktivieren wenn mit WireGuard gearbeitet wird
# wg_ip=$(ip -f inet addr show wg0 2>/dev/null | grep -Po 'inet \K[\d.]+') 
# [ "$wg_ip" != "" ] && { echo $wg_ip >~/work/server-ip; }

echo $public_ip

- - -

# UUID

Die nachfolgende Zeile generiert eine eindeutige UUID (Universally Unique Identifier).

Die UUID dient als unverwechselbare Kennung, die sicherstellt, dass jede Installation eindeutig identifiziert werden kann. 

Damit kann, z.B. für die IoT Übungen ein eindeutiges MQTT Topic bereitgestellt werden.

Die generierte UUID wird im Python-Dateiformat uuid.py im Verzeichnis ~/work/ gespeichert und wird in den Python-Skripten weiterverwendet.

In [ ]:
! echo "UUID=\"$(uuid)\"" >~/work/uuid.py

- - -

# Docker.io Login 

Um die Pull-Raten-Limits von Docker zu umgeben, Melden wir uns bei Docker.io an und hinterlegen die Anmeldeinformationen in Kubernetes.


In [ ]:
%%bash
for ns in $(microk8s kubectl get ns --no-headers -o custom-columns=":metadata.name"); do
  microk8s kubectl create secret docker-registry regcred \
    --docker-server=https://index.docker.io/v1/ \
    --docker-username=misegr \
    --docker-password='dckr_pat_Su9Wt8rqpxC2Wixm7_token1234' \
    --namespace $ns
done
for ns in $(microk8s kubectl get ns --no-headers -o custom-columns=":metadata.name"); do
  microk8s kubectl patch serviceaccount default \
    -p '{"imagePullSecrets":[{"name":"regcred"}]}' \
    -n $ns
done


- - - 

Pull-Raten-Limits
=================
Docker Hub verwendet IP-Adressen, um die Benutzer zu authentifizieren, und Pull-Raten-Limits basieren auf einzelnen IP-Adressen. 

Für **anonyme Benutzer** ist das Ratenlimit auf 100 Abrufe pro 6 Stunden pro IP-Adresse festgelegt.

Die aktuellen Zugriff können wir wie folgt abfragen:

In [ ]:
%%bash
TOKEN=$(curl -s "https://auth.docker.io/token?service=registry.docker.io&scope=repository:ratelimitpreview/test:pull" | jq -r .token)
curl --head -H "Authorization: Bearer $TOKEN" https://registry-1.docker.io/v2/ratelimitpreview/test/manifests/latest

Holen der Container Images von einer Alternativen Container Registry
==

In [ ]:
%%bash
#!/bin/bash
# pull-alternative-images.sh
# Lädt alternative Images, taggt sie als Ersatz für docker.io

declare -A map=(
  ["docker.io/calico/cni:v3.28.1"]="quay.io/calico/cni:v3.28.1"
  ["docker.io/calico/kube-controllers:v3.28.1"]="quay.io/calico/kube-controllers:v3.28.1"
  ["docker.io/calico/node:v3.28.1"]="quay.io/calico/node:v3.28.1"
  ["docker.io/coredns/coredns:1.10.1"]="registry.k8s.io/coredns/coredns:1.10.1"
  ["docker.io/istio/pilot:1.24.2"]="gcr.io/istio-release/pilot:1.24.2"
  ["docker.io/istio/proxyv2:1.24.2"]="gcr.io/istio-release/proxyv2:1.24.2"
  ["docker.io/kubernetesui/dashboard:v2.7.0"]="registry.gitlab.com/mc-b/misegr/kubernetesui/dashboard:v2.7.0"
  ["docker.io/openzipkin/zipkin-slim:3.4.0"]="ghcr.io/openzipkin/zipkin-slim:3.4.0"
)

for orig in "${!map[@]}"; do
  alt="${map[$orig]}"
  echo "Pulling alternative: $alt"
  microk8s ctr images pull "$alt"
  echo "Tagging as $orig"
  microk8s ctr images tag "$alt" "$orig" || true
done
